# Debugging DRP Outputs
This page describes several techniques to find problems and inconsistencies from DRP Processing.

The first step is to load a copy of the time series database.

In [1]:
from modules.quicklook.src.analyze_time_series import AnalyzeTimeSeries
from datetime import datetime
import pandas as pd
pd.set_option('display.max_rows', None) # show all rows
pd.set_option('display.max_columns', None)  # show all columns without wrapping
pd.set_option('display.width', 200)
db_path = 'kpf_ts.db'
myTS = AnalyzeTimeSeries(db_path=db_path)

INFO: Starting AnalyzeTimeSeries
INFO: Starting KPF_TSDB
INFO: Jupyter Notebook environment detected.
INFO: Path of database file: /code/KPF-Pipeline/docs/source/tutorials/kpf_ts.db
INFO: Base data directory: /data/L0
INFO: Primary tables already exist.


In [2]:
if False:
    myTS = AnalyzeTimeSeries(db_path=db_path)
    myTS.db.print_summary_all_tables()
    #myTS.db.drop_tables()
myTS = AnalyzeTimeSeries(db_path=db_path)

INFO: Starting AnalyzeTimeSeries
INFO: Starting KPF_TSDB
INFO: Jupyter Notebook environment detected.
INFO: Path of database file: /code/KPF-Pipeline/docs/source/tutorials/kpf_ts.db
INFO: Base data directory: /data/L0
INFO: Primary tables already exist.


In [3]:
myTS.db.print_summary_all_tables()

Table Name           Columns       Rows
----------------------------------------
tsdb_2d                   84       1921
tsdb_base                 11       1921
tsdb_l0                   87       1921
tsdb_l0t                 124       1921
tsdb_l1                  523       1921
tsdb_l2                   17       1921
tsdb_l2ccf                 7       1921
tsdb_l2rv                 29       1921
tsdb_l2rvdata            805       1921
tsdb_metadata              6       1679


In [4]:
if False:
    start_date = '20250309'
    end_date   = '20250315'
    myTS.db.ingest_dates_to_db(start_date, end_date)

In [5]:
myTS.db.print_metadata_table()

Base Keywords:
------------------------------------------------------------------------------------------------------------------------------------------------------
Keyword                             Datatype  Units     Description                                                                               
------------------------------------------------------------------------------------------------------------------------------------------------------
D2_filename                         string              Full path to 2D file                                                                      
D2_header_read_time                 string              Last modification of 2D file                                                              
L0_filename                         string              Full path to L0 file                                                                      
L0_header_read_time                 string              Last modification of L0 file           

## Finding Outlier Observations
The code below returns a dataframe with useful keywords in a date range.  `not_junk` can be set to True or False depending on the desired outcome.

In [6]:
start_date = datetime(2025,  3,  1)
end_date   = datetime(2025, 3, 31)
columns_to_display = ['ObsID', 'OBJECT', 'Source', 'kpfgreen.STA_CCD_T', 'kpfred.STA_CCD_T', 'kpfgreen.KPF_CCD_T', 'kpfred.KPF_CCD_T']
df = myTS.db.dataframe_from_db(columns_to_display, start_date=start_date, end_date=end_date, verbose=True)

DEBUG: SQL Query:
DEBUG: 
                SELECT tsdb_l0."FIUMODE" AS "FIUMODE", tsdb_l0."ObsID" AS "ObsID", tsdb_l0."OBJECT" AS "OBJECT", tsdb_base."Source" AS "Source", tsdb_l0t."kpfgreen.STA_CCD_T" AS "kpfgreen.STA_CCD_T", tsdb_l0t."kpfgreen.KPF_CCD_T" AS "kpfgreen.KPF_CCD_T", tsdb_l0t."kpfred.STA_CCD_T" AS "kpfred.STA_CCD_T", tsdb_l0t."kpfred.KPF_CCD_T" AS "kpfred.KPF_CCD_T", tsdb_2d."NOTJUNK" AS "NOTJUNK"
                FROM tsdb_base
                LEFT JOIN tsdb_l0 ON tsdb_base."ObsID" = tsdb_l0."ObsID" LEFT JOIN tsdb_l0t ON tsdb_base."ObsID" = tsdb_l0t."ObsID" LEFT JOIN tsdb_2d ON tsdb_base."ObsID" = tsdb_2d."ObsID"
                WHERE tsdb_base."datecode" >= ? AND tsdb_base."datecode" <= ?
            
DEBUG: Params:
DEBUG: ['20250301', '20250331']


In [9]:
#print(df)

In [14]:
cond = ['kpfgreen.STA_CCD_T < -110']
df = myTS.db.dataframe_from_db(columns_to_display, start_date=start_date, end_date=end_date, verbose=True, extra_conditions=cond, extra_conditions_logic='AND')
                          

DEBUG: SQL Query:
DEBUG: 
                SELECT tsdb_l0."FIUMODE" AS "FIUMODE", tsdb_l0."ObsID" AS "ObsID", tsdb_l0."OBJECT" AS "OBJECT", tsdb_base."Source" AS "Source", tsdb_l0t."kpfgreen.STA_CCD_T" AS "kpfgreen.STA_CCD_T", tsdb_l0t."kpfgreen.KPF_CCD_T" AS "kpfgreen.KPF_CCD_T", tsdb_l0t."kpfred.STA_CCD_T" AS "kpfred.STA_CCD_T", tsdb_l0t."kpfred.KPF_CCD_T" AS "kpfred.KPF_CCD_T", tsdb_2d."NOTJUNK" AS "NOTJUNK"
                FROM tsdb_base
                LEFT JOIN tsdb_l0 ON tsdb_base."ObsID" = tsdb_l0."ObsID" LEFT JOIN tsdb_l0t ON tsdb_base."ObsID" = tsdb_l0t."ObsID" LEFT JOIN tsdb_2d ON tsdb_base."ObsID" = tsdb_2d."ObsID"
                WHERE tsdb_base."datecode" >= ? AND tsdb_base."datecode" <= ? AND (tsdb_l0t."kpfgreen.STA_CCD_T" < -110)
            
DEBUG: Params:
DEBUG: ['20250301', '20250331']


In [15]:
print(df)

                  ObsID             OBJECT Source  kpfgreen.STA_CCD_T  kpfred.STA_CCD_T  kpfgreen.KPF_CCD_T  kpfred.KPF_CCD_T
0  KP.20250309.71857.86  bias-noise-test-2   Bias             -273.15           -273.15             -273.15           -273.15
1  KP.20250309.71907.72  bias-noise-test-2   Bias             -273.15           -273.15             -273.15           -273.15
2  KP.20250309.71957.77  bias-noise-test-2   Bias             -273.15           -273.15             -273.15           -273.15
3  KP.20250309.72007.76  bias-noise-test-2   Bias             -273.15           -273.15             -273.15           -273.15
4  KP.20250309.72057.69  bias-noise-test-2   Bias             -273.15           -273.15             -273.15           -273.15
5  KP.20250309.72118.35  bias-noise-test-2   Bias             -273.15           -273.15             -273.15           -273.15
6  KP.20250309.72168.39  bias-noise-test-2   Bias             -273.15           -273.15             -273.15           